## 1. Frame the problem 

In [1]:
from __future__ import print_function                  # Allows for python3 printing
import keras
from keras.datasets import cifar100                    # import function to load cifar100 dataset
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten,Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU

c:\users\asus\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 2. Get the data

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

## 3. Explore the data

## 4. Prepare the data

In [3]:
x_train = x_train/255.0
x_test = x_test/255.0
y_train = keras.utils.to_categorical(y_train[:,0], num_classes=100)
y_test = keras.utils.to_categorical(y_test[:,0], num_classes=100)

## 5.1 Create the model

In [4]:
def ConvNet(num_classes=100):

    images = Input(shape=(32, 32, 3))           #use input shape that matches image size
    
    # Convolutional layers
    x = Conv2D(filters=64, kernel_size=(4, 4), padding='valid')(images)  #conv layer1
    LeakyReLU(alpha=0.3) #Using LeakyReLU as activation
    x = Dropout(0.25)(x)                     #dropout layer1
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(images)  #conv layer2
    x = Dropout(0.20)(x)                     #dropout layer2
    
    x = Conv2D(filters=64, kernel_size=(2, 2), activation='relu')(x)   #conv layer3
    x = MaxPooling2D(pool_size=(2, 2))(x)  #pooling layer2
    x = Dropout(0.5)(x)                     #dropout layer3

    # Fully-connected layer: 
    x = Flatten()(x)          
    x = Dense(200)(x)         
    x = Dense(units=num_classes, activation='softmax')(x)

    # Instantiate a model based on inputs/outputs
    model = Model(inputs=images, outputs=x)

    # Configure the model for training. 
    model.compile(
            loss='categorical_crossentropy',
            optimizer=optimizers.RMSprop(lr=0.001), #using RMSprop optimizer
            metrics=[
                'categorical_accuracy',
                keras.metrics.top_k_categorical_accuracy])

    return model

## 5.2 train the model

In [5]:
model = ConvNet()
##Using early stop
best_weights_filepath = './best_weights.hdf5' 
                                            
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', 
                                              patience = 2) 

SaveBestWeights = keras.callbacks.ModelCheckpoint(best_weights_filepath,
                                                  monitor='val_acc',
                                                  save_best_only=True)
model.fit(
        x=x_train,
        y=y_train,
        epochs=30,
        shuffle=True)


Epoch 1/30
50000/50000 [==============================] - 399s 8ms/step - loss: 3.5304 - categorical_accuracy: 0.1814 - top_k_categorical_accuracy: 0.4194
Epoch 2/30
50000/50000 [==============================] - ETA: 0s - loss: 2.8332 - categorical_accuracy: 0.3107 - top_k_categorical_accuracy: 0.603 - 404s 8ms/step - loss: 2.8332 - categorical_accuracy: 0.3107 - top_k_categorical_accuracy: 0.6039
Epoch 3/30
50000/50000 [==============================] - 403s 8ms/step - loss: 2.4986 - categorical_accuracy: 0.3774 - top_k_categorical_accuracy: 0.6748
Epoch 4/30
50000/50000 [==============================] - 402s 8ms/step - loss: 2.2701 - categorical_accuracy: 0.4240 - top_k_categorical_accuracy: 0.7238
Epoch 5/30
50000/50000 [==============================] - 387s 8ms/step - loss: 2.0895 - categorical_accuracy: 0.4615 - top_k_categorical_accuracy: 0.7586
Epoch 6/30
50000/50000 [==============================] - 412s 8ms/step - loss: 1.9437 - categorical_accuracy: 0.4925 - top_k_categor

## 6. Present your model

In [6]:
train_score = model.evaluate(x_train, y_train)      
test_score = model.evaluate(x_test, y_test)          

print ('\n')                                              
print('Training accuracy:', train_score[1])          
print('Testing accuracy:', test_score[1])            

10000/10000 [==============================] - 17s 2ms/step


Training accuracy: 0.94114
Testing accuracy: 0.3694
